<a href="https://colab.research.google.com/github/andriasmagno/andriasmagno-YouTube-Insights-Extractor/blob/main/Machine_learning_with_You_Tube_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Caminho do arquivo CSV (ajuste conforme o caminho onde o arquivo está salvo)
file_path = r''

# Função para converter visualizações e comentários para números
def convert_to_numeric(value):
    value = value.replace('K', 'e3').replace('M', 'e6')
    return pd.to_numeric(value)

# Ler o arquivo CSV usando pandas
df = pd.read_csv(file_path)

# Função para calcular Visualizações por Unidade de Tempo (VUT)
def calculate_vut(df):
    df['Days Since Published'] = (pd.to_datetime('today') - pd.to_datetime(df['Date'])).dt.days
    df['Views'] = df['Views'].apply(convert_to_numeric)
    df['VUT'] = df['Views'] / df['Days Since Published']
    return df

# Função para calcular Engajamento Relativo (ER)
def calculate_engagement_rate(df):
    df['Comments'] = df['Comments'].apply(convert_to_numeric)
    df['ER'] = df['Comments'] / df['Views']
    return df

# Função para calcular Estimativa de Ganho por Unidade de Tempo (EGUT)
def calculate_egut(df):
    df['Average Earnings'] = (df['Earnings (Low)'].str.replace('$', '').str.replace('K', 'e3').str.replace('M', 'e6').apply(pd.to_numeric) +
                              df['Earnings (High)'].str.replace('$', '').str.replace('K', 'e3').str.replace('M', 'e6').apply(pd.to_numeric)) / 2
    df['EGUT'] = df['Average Earnings'] / df['Days Since Published']
    return df

# Função para calcular a Relação Visualizações/Ganhos (V/G)
def calculate_views_per_gain(df):
    df['VG'] = df['Views'] / df['Average Earnings']
    return df

# Função para normalizar as métricas
def normalize_column(column):
    return (column - column.min()) / (column.max() - column.min())

# Função para calcular a pontuação final
def calculate_scores(df):
    # Normalizar as métricas
    df['VUT_normalized'] = normalize_column(df['VUT'])
    df['ER_normalized'] = normalize_column(df['ER'])
    df['EGUT_normalized'] = normalize_column(df['EGUT'])
    df['VG_normalized'] = normalize_column(df['VG'])

    # Criar uma pontuação final (média das métricas normalizadas)
    df['Score'] = (df['VUT_normalized'] + df['ER_normalized'] + df['EGUT_normalized'] + df['VG_normalized']) / 4

    return df

# Aplicar as funções e gerar as métricas
df = calculate_vut(df)
df = calculate_engagement_rate(df)
df = calculate_egut(df)
df = calculate_views_per_gain(df)

# Calcular as pontuações para cada vídeo
df = calculate_scores(df)

# Transformar a pontuação em uma classe binária (1 para vídeos "bons" e 0 para "ruins")
df['Target'] = (df['Score'] > df['Score'].median()).astype(int)

# Separar as variáveis preditoras (features) e a variável alvo (target)
X = df[['VUT', 'ER', 'EGUT', 'VG']]
y = df['Target']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy:.2f}')
print("Relatório de classificação:")
print(classification_report(y_test, y_pred))

# Função para prever e exibir os detalhes de um novo vídeo
def predict_new_video(vut, er, egut, vg):
    new_video = pd.DataFrame([[vut, er, egut, vg]], columns=['VUT', 'ER', 'EGUT', 'VG'])
    prediction = clf.predict(new_video)

    # Printar as métricas do vídeo testado
    print(f"Vídeo testado:")
    print(f"VUT (Visualizações por Unidade de Tempo): {vut}")
    print(f"ER (Engajamento Relativo): {er}")
    print(f"EGUT (Estimativa de Ganho por Unidade de Tempo): {egut}")
    print(f"VG (Relação Visualizações/Ganhos): {vg}")

    # Exibir o resultado da previsão
    resultado = "Bom" if prediction[0] == 1 else "Ruim"
    print(f"Resultado da previsão: O vídeo é considerado: {resultado}")
    return resultado

# Exemplo de previsão para um novo vídeo com métricas fictícias
vut = 10000  # Visualizações por unidade de tempo
er = 0.05    # Engajamento relativo
egut = 200   # Estimativa de ganho por unidade de tempo
vg = 50      # Relação Visualizações/Ganhos

resultado = predict_new_video(vut, er, egut, vg)
